## LBC group management


In [32]:
import collections
from dataclasses import dataclass
import pandas as pd
import numpy as np
from collections import namedtuple
group_chars = [c for c in 'ABCDEFGHIJKLMNOP']
requirement = namedtuple('requirement', 'target min max')
sc_group_size = requirement(8,6,9)
sc_singles_count = requirement(2,0,4)
sc_couple_count = requirement(3,3,4)
lb_group_size = requirement(6,5,7)
lb_singles_count = requirement(2,0,3)
lb_couple_count = requirement(2,2,3)


### import historical groupings
- columns are groupings, of either type 'lb' or 'sc'
- rows are members
- character denotes membership in a group. same letter same group. null for no involvement

In [33]:
column_names = ['size'] + list(f'{"sc" if i%2 else "lb"}{i//2}'
                             for i in range(29)
                             ) + ['lb_block', 'sc_block']
group_history = pd.read_csv("LBC_group_history_2.csv", names=column_names,header=0)

display(group_history.head(5))

,size,lb0,sc0,lb1,sc1,lb2,sc2,lb3,sc3,lb4,...,sc10,lb11,sc11,lb12,sc12,lb13,sc13,lb14,lb_block,sc_block
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,NaN,NaN,NaN,NaN,NaN,A,NaN,X,NaN
1,1,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X
2,2,D,NaN,F,NaN,G,NaN,NaN,B,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X
3,1,A,E,F,C,I,B,F,C,F,...,A,NaN,NaN,NaN,C,NaN,D,NaN,X,NaN
4,2,NaN,E,E,NaN,NaN,A,NaN,NaN,C,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,X,X


### measure group sizes
- get total group sizes using weights from size column


In [34]:
weights = group_history.iloc[:, 0].values

group_sizes = pd.DataFrame([
    [
        np.sum([weights[i] if value == group_char else 0 for i, value in enumerate(group_history[column])])
        for group_char in group_chars
    ] for column in group_history.columns[1:-2]
],
columns=group_chars,
index=[column for column in group_history.columns[1:-2]],
).replace(0, np.nan)

display(group_sizes.tail(5))


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
lb12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sc12,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lb13,4.0,6.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sc13,8.0,8.0,10.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lb14,9.0,6.0,4.0,6.0,6.0,6.0,6.0,6.0,7.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN


### create met_before lookup dict
- key: group history index
- value: list of group history indexes


In [35]:
all_groups = list()
for col_name, grouping in group_history.iteritems():
    for group_char in set(grouping):
        if group_char in group_chars:
            all_groups.append(group_history.index[group_history[col_name] == group_char].tolist())

met_before = {i: set([i,]) for i, groups in group_history.iterrows()}
for group in all_groups:
    for i in group:
        for j in group:
            met_before[i].add(j)



### data sanity check
- expect 10-25 unique groups in each round
- groups around 4-8 in size
- max possible group memberships is 29
- 20-50% of members never active before


In [36]:
# filter row data
# [list(filter(lambda x: x>0, grouping)) for i, grouping in group_sizes.iterrows()]

group_counts = [np.sum(len(list(filter(lambda x: x>0, grouping)))) for i, grouping in group_sizes.iterrows()]

features = pd.DataFrame(
    [
        [
            # number of groups
            np.mean(group_counts),
            np.min(group_counts),
            np.max(group_counts),
        ],
        [
            # group size
            np.mean([np.nanmean(group_sizes)]),
            np.min([np.nanmin(group_sizes)]),
            np.max([np.nanmax(group_sizes)]),
        ],
        [
            # socializing
            np.mean([len(met_set) for i, met_set in met_before.items()]),
            np.min([len(met_set) for i, met_set in met_before.items()]),
            np.max([len(met_set) for i, met_set in met_before.items()]),
        ],
    ],
    columns=['mean', 'min', 'max'],
    index=['number of groups', 'group size head count', 'met counts'],
)

display(features)

,mean,min,max
number of groups,4.448276,0.0,11.0
group size head count,5.906977,1.0,10.0
met counts,11.000000,1.0,46.0


# Create next SC and LB groupings
- confirm group requirements


In [37]:
display(pd.DataFrame([
    sc_group_size, sc_singles_count, sc_couple_count,
    lb_group_size, lb_singles_count, lb_couple_count,
],
    columns=['target', 'min', 'max'],
    index=['sc group size', 'sc singles count', 'sc couple count',
           'lb group size', 'lb singles count', 'lb couple count',],
))

,target,min,max
sc group size,8,6,9
sc singles count,2,0,4
sc couple count,3,3,4
lb group size,6,5,7
lb singles count,2,0,3
lb couple count,2,2,3


- create groups
- disperse indexes by order of met counts
- check member requirements
- check total size limits

In [38]:
# sort indexes by weight (couples first), then met_before count (most first_
ordered_indexes = collections.OrderedDict(sorted(met_before.items(), key=lambda met_set: (-weights[met_set[0]], -len(met_set[1]))))
# for i, met_set in ordered_indexes.items():
#     print(f"{i} has met {met_set}")

In [39]:
indexes_list = list(ordered_indexes.keys())

min_groups_count = {
    "sc": 7,  # np.sum(weights) / sc_group_size.target
    "lb": 10,  # np.sum(weights) / lb_group_size.max
}

@dataclass
class NewGroup:
    members: list
    singles: int
    couples: int
    size: int
    kind: str

def i_can_join(asking_index: int, new_group: NewGroup, please=False):
    if asking_index in new_group.members:
        raise Exception("already a member of this group!")

    if new_group.size >= (lb_group_size.max if new_group.kind=="lb" else sc_group_size.max):
        return False

    if weights[asking_index] == 1 and \
            new_group.singles >= (lb_singles_count.max if new_group.kind=="lb" else sc_singles_count.max):
        return False

    if weights[asking_index] == 2 and \
            new_group.couples >= (lb_couple_count.max if new_group.kind=="lb" else sc_couple_count.max):
        return False

    if please != "pretty please":
        if len(set(new_group.members).intersection(met_before[asking_index])) > 2:
            return False

    if not please:
        if len(set(new_group.members).intersection(met_before[asking_index])) > 0:
            return False

        if new_group.size >= (lb_group_size.target if new_group.kind=="lb" else sc_group_size.target):
            return False
        if weights[asking_index] == 1 and new_group.singles >= (lb_singles_count.target if new_group.kind=="lb" else sc_singles_count.target):
            return False
        if weights[asking_index] == 2 and new_group.couples >= (lb_couple_count.target if new_group.kind=="lb" else sc_couple_count.target):
            return False

    if len(set(new_group.members).intersection(met_before[asking_index])) > 0:
        those_meeting_the_met = set(new_group.members).intersection(met_before[asking_index])
        print(f'index {i} has met with {those_meeting_the_met} and will see them again')

    return True


def populate_grouping(grouping, group_kind):
    placed_indexes = set()
    displaced_indexes = set()
    for index in indexes_list:
        this_index_placed = False

        if "X" == group_history[f"{group_kind}_block"].values[index]:
            # print(f'index {index} found {group_history[f"{group_kind}_block"].values[index]}')
            continue

        for group_i, group in enumerate(grouping):

            #  fill an empty group
            if len(group.members) == 0:
                assert(group.size==0)
                grouping[group_i].members.append(index)
                this_index_placed = True
                break

            # join group if not full and unmet members
            if i_can_join(index, group):
                grouping[group_i].members.append(index)
                this_index_placed = True
                break

            # create a new group
            if len(grouping) < min_groups_count[group_kind]:
                grouping.append(NewGroup([index,], 0, 0, 0, group_kind))
                this_index_placed = True
                group_i = len(grouping)-1
                break

        # uncomment to allow going beyond targets
        if not this_index_placed:
            for group_i, group in enumerate(grouping):
                if i_can_join(index, group, please=True):  # can set please="pretty please"
                    grouping[group_i].members.append(index)
                    this_index_placed = True
                    break

        if this_index_placed:
            grouping[group_i].size += weights[index]
            if weights[index] == 1:
                grouping[group_i].singles += 1
            else:
                grouping[group_i].couples += 1
            placed_indexes.add(index)
        else:
            displaced_indexes.add(index)

    return grouping, displaced_indexes

lb_groupings = [NewGroup([],0,0,0,"lb"),]
lb_groupings, lb_unfit_list = populate_grouping(lb_groupings, "lb")

sc_groupings = [NewGroup([],0,0,0,"sc"),]
sc_groupings, sc_unfit_list = populate_grouping(sc_groupings, "sc")

index 80 has met with {79} and will see them again
index 80 has met with {67} and will see them again
index 80 has met with {107} and will see them again
index 80 has met with {107} and will see them again


# Results when being strict with targets

In [40]:
print(f"LB needed at least {int(min_groups_count['lb'])} groups. and ended up creating {len(lb_groupings)} groups")
display(lb_groupings)
if lb_unfit_list:
    print(f"{len(lb_unfit_list)} leftover could not be placed")
else:
    print("Everyone was placed!")

LB needed at least 10 groups. and ended up creating 10 groups


[NewGroup(members=[107, 67, 58, 106], singles=1, couples=3, size=7, kind='lb'),
 NewGroup(members=[33, 79, 59], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[61, 105, 73], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[76, 20, 74], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[110, 82, 95], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[53, 30, 124], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[112, 116, 127], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[90, 108, 99, 94], singles=2, couples=2, size=6, kind='lb'),
 NewGroup(members=[54, 6, 70, 126], singles=2, couples=2, size=6, kind='lb'),
 NewGroup(members=[80, 51, 45, 15], singles=2, couples=2, size=6, kind='lb')]

Everyone was placed!


In [41]:
print(f"SC needed at least {int(min_groups_count['sc'])} groups. and ended up creating {len(sc_groupings)} groups")
display(sc_groupings)
if sc_unfit_list:
    print(f"{len(sc_unfit_list)} leftover could not be placed")
else:
    print("Everyone was placed!")

SC needed at least 7 groups. and ended up creating 7 groups


[NewGroup(members=[107, 80, 123, 19, 45], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[93, 113, 79, 8, 125], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[76, 39, 82, 99, 64], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[110, 116, 108, 70], singles=1, couples=3, size=7, kind='sc'),
 NewGroup(members=[89, 30, 42, 15], singles=1, couples=3, size=7, kind='sc'),
 NewGroup(members=[96, 85, 92, 109, 50], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[0, 127, 128, 3], singles=1, couples=3, size=7, kind='sc')]

Everyone was placed!


# Results when being loose with targets

In [42]:
print(f"LB needed at least {int(min_groups_count['lb'])} groups. and ended up creating {len(lb_groupings)} groups")
display(lb_groupings)
if lb_unfit_list:
    print(f"{len(lb_unfit_list)} leftover could not be placed")
else:
    print("Everyone was placed!")

LB needed at least 10 groups. and ended up creating 10 groups


[NewGroup(members=[107, 67, 58, 106], singles=1, couples=3, size=7, kind='lb'),
 NewGroup(members=[33, 79, 59], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[61, 105, 73], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[76, 20, 74], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[110, 82, 95], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[53, 30, 124], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[112, 116, 127], singles=0, couples=3, size=6, kind='lb'),
 NewGroup(members=[90, 108, 99, 94], singles=2, couples=2, size=6, kind='lb'),
 NewGroup(members=[54, 6, 70, 126], singles=2, couples=2, size=6, kind='lb'),
 NewGroup(members=[80, 51, 45, 15], singles=2, couples=2, size=6, kind='lb')]

Everyone was placed!


In [43]:
print(f"SC needed at least {int(min_groups_count['sc'])} groups. and ended up creating {len(sc_groupings)} groups")
display(sc_groupings)
if sc_unfit_list:
    print(f"{len(sc_unfit_list)} leftover could not be placed")
else:
    print("Everyone was placed!")

SC needed at least 7 groups. and ended up creating 7 groups


[NewGroup(members=[107, 80, 123, 19, 45], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[93, 113, 79, 8, 125], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[76, 39, 82, 99, 64], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[110, 116, 108, 70], singles=1, couples=3, size=7, kind='sc'),
 NewGroup(members=[89, 30, 42, 15], singles=1, couples=3, size=7, kind='sc'),
 NewGroup(members=[96, 85, 92, 109, 50], singles=2, couples=3, size=8, kind='sc'),
 NewGroup(members=[0, 127, 128, 3], singles=1, couples=3, size=7, kind='sc')]

Everyone was placed!


In [45]:
jean = 4
print("all the people Jean met before")
print(sorted(list(met_before[jean])))

all the people Jean met before
[3, 4, 17, 25, 39, 43, 49, 57, 62, 70, 71, 87, 93, 96, 99, 100, 122]
